# RAG Evaluation
_Authored by: [Aymeric Roucher](https://huggingface.co/m-ric)_

This notebook demonstrates how you can evaluate your RAG (Retrieval Augmented Generation), by building a synthetic evaluation dataset and using LLM-as-a-judge to compute the accuracy of your system.

For an introduction to RAG, you can check [this other cookbook](rag_zephyr_langchain)!

RAG systems are complex: here a RAG diagram, where we noted in blue all possibilities for system enhancement:

<img src="https://huggingface.co/datasets/huggingface/cookbook-images/resolve/main/RAG_workflow.png" height="700">

Implementing any of these improvements can bring a huge performance boost; but changing anything is useless if you cannot monitor the impact of your changes on the system's performance!
So let's see how to evaluate our RAG system.

### Evaluating RAG performance

Since there are so many moving parts to tune with a big impact on performance, benchmarking the RAG system is crucial.

For our evaluation pipeline, we will need:
1. An evaluation dataset with question - answer couples (QA couples)
2. An evaluator to compute the accuracy of our system on the above evaluation dataset.

➡️ It turns out, we can use LLMs to help us all along the way!
1. The evaluation dataset will be synthetically generated by an LLM 🤖, and questions will be filtered out by other LLMs 🤖
2. An [LLM-as-a-judge](https://huggingface.co/papers/2306.05685) agent 🤖 will then perform the evaluation on this synthetic dataset.

__Let's dig into it and start building our evaluation pipeline!__ First, we install the required model dependancies.

In [ ]:
#!pip install -q torch transformers transformers langchain sentence-transformers tqdm openpyxl openai pandas datasets

In [ ]:
# %reload_ext autoreload
# %autoreload 2

In [1]:
from tqdm.auto import tqdm
import pandas as pd
from typing import Optional, List, Tuple
import json
import datasets

pd.set_option("display.max_colwidth", None)

In [2]:
from huggingface_hub import notebook_login

notebook_login()

### Load your knowledge base

In [61]:
ds = datasets.load_dataset("m-ric/huggingface_doc", split="train")


# 1. Build a synthetic dataset for evaluation

### 1.1. Prepare source documents

- Drop REFERENCES onward, respecting previous discoveries that these just introduce noise to retrieval.  Must process new docs the same way!

In [10]:
import os
import PyPDF2
from PyPDF2 import PdfReader
from pathlib import Path
PDFS_PATH = Path('/home/mainuser/Desktop/LLMs/RagOverArXiv/clusterofstars')
PDFS = list(PDFS_PATH.glob('*.pdf'))
PDFS[0], len(PDFS)

reader = PdfReader(os.path.expanduser(PDFS[0]))
pages = reader.pages
documents = []
for page in pages:
  documents.append(page.extract_text())


def load_pdf_to_string(pdf_path):
    # Open the PDF file in binary mode
    with open(pdf_path, 'rb') as file:
        # Create a PDF file reader object
        pdf_reader = PyPDF2.PdfReader(file)

        # Initialize an empty string to hold the text
        text = ''

        # Loop through each page and extract the text
        for page_num in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_num]
            page_text = page.extract_text()
            references_index= page_text.upper().find('\nREFERENCES\n')
            if references_index != -1:
              page_text = page_text[:references_index]
              text += page_text
              return text
            text += page_text
    return text

# Use the function to load a PDF into a string
text = load_pdf_to_string(os.path.expanduser(PDFS[0]))
def get_title(pdf_path): return os.path.expanduser(pdf_path).split('/')[-1]

all_docs_and_titles = [(load_pdf_to_string(os.path.expanduser(pdf_path)),get_title(pdf_path)) for pdf_path in PDFS]

all_docs = [doc[0] for doc in all_docs_and_titles]
all_titles = [doc[1] for doc in all_docs_and_titles]

from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter, TokenTextSplitter, RecursiveCharacterTextSplitter
from langchain.docstore.document import Document 

# CHUNK_SIZE = 1000 #try 2000 next
# CHUNK_OVERLAP = 30 #try 200 next
CHUNK_SIZE = 2000 #try 2000 next
CHUNK_OVERLAP = 200 #try 200 next

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap = CHUNK_OVERLAP,
    length_function=len,
)

docs_processed  = [text_splitter.split_documents([Document(page_content=doc, metadata={'source':all_titles[idx]})]) 
         for idx,doc in enumerate(all_docs)]

# text_splitter = RecursiveCharacterTextSplitter(
#     chunk_size=2000,
#     chunk_overlap=200,
#     add_start_index=True,
#     #separators=["\n\n", "\n", ".", " ", ""],
# )

# docs_processed = []
# for idx,doc in enumerate(all_docs):
#     docs_processed += text_splitter.split_documents([Document(page_content=doc, metadata={'source':all_titles[idx]})])


In [11]:
docs_processed = [txt for doc in docs_processed for txt in doc]

In [12]:
len(docs_processed)

700

In [13]:
docs_processed[0]

Document(page_content='CHAIN -OF-VERIFICATION REDUCES HALLUCINATION\nINLARGE LANGUAGE MODELS\nShehzaad Dhuliawala\nMeta AI & ETH Z ¨urichMojtaba Komeili\nMeta AIJing Xu\nMeta AIRoberta Raileanu\nMeta AI\nXian Li\nMeta AIAsli Celikyilmaz\nMeta AIJason Weston\nMeta AI\nABSTRACT\nGeneration of plausible yet incorrect factual information, termed hallucination,\nis an unsolved issue in large language models. We study the ability of language\nmodels to deliberate on the responses they give in order to correct their mistakes.\nWe develop the Chain-of-Verification (C OVE) method whereby the model first (i)\ndrafts an initial response; then (ii) plans verification questions to fact-check its\ndraft; (iii) answers those questions independently so the answers are not biased\nby other responses; and (iv) generates its final verified response. In experiments,\nwe show COVEdecreases hallucinations across a variety of tasks, from list-based\nquestions from Wikidata, closed book MultiSpanQA and longfo

- Make embeddings and save to vector store

In [54]:
from langchain.embeddings import CacheBackedEmbeddings, HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore

store = LocalFileStore("./cache/")

embed_model_id = 'sentence-transformers/all-MiniLM-L6-v2'

core_embeddings_model = HuggingFaceEmbeddings(
    model_name=embed_model_id
)

embedder = CacheBackedEmbeddings.from_bytes_store(
    core_embeddings_model, store, namespace=embed_model_id
)


from langchain.schema.document import Document

#docs = [Document(page_content=doc[i],metadata={'source':all_titles[j]}) for j,doc in enumerate(docs) for i in range(len(doc))]
for index, pdf in enumerate(docs_processed):
   content = docs_processed[index]
   if index == 0:
       vector_store = FAISS.from_documents([content], embedder)
   else:
      vector_store_i = FAISS.from_documents([content], embedder)
      vector_store.merge_from(vector_store_i)

vector_store


model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

#### TODO: Once have the pipeline working and have a baseline, look into https://huggingface.co/spaces/mteb/leaderboard.  SFR-Embedding-Mistral or something along those lines may work much better.

In [57]:
vector_store.save_local('../rag_index_dir')

### 1.2. Setup agents for question generation

- HF used [Mixtral](https://huggingface.co/mistralai/Mixtral-8x7B-Instruct-v0.1) for QA couple generation because it it has excellent performance in leaderboards such as [Chatbot Arena](https://huggingface.co/spaces/lmsys/chatbot-arena-leaderboard).  Used Mixtral-8x7b-4bit exl2, and it did not appear significantly better than Mistral, so using Mistral for speed but may come back to this.

In [67]:
# from huggingface_hub import InferenceClient


# repo_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"

# llm_client = InferenceClient(
#     model=repo_id,
#     timeout=120,
# )


# def call_llm(inference_client: InferenceClient, prompt: str):
#     response = inference_client.post(
#         json={
#             "inputs": prompt,
#             "parameters": {"max_new_tokens": 1000},
#             "task": "text-generation",
#         },
#     )
#     return json.loads(response.decode())[0]["generated_text"]


# call_llm(llm_client, "This is a test context")

#### TODO: Tried Mixtral4Bit, visually perhaps a bit better, but overfits on 'deep question'. Perhaps return to this after looking at embedding model

In [6]:
from exllamav2 import *
from exllamav2.generator import *
import sys, torch


generator_config = ExLlamaV2Config()
generator_config.model_dir = "/home/mainuser/Desktop/LLMs/MiStralInference"
#generator_config.model_dir = '/home/mainuser/Desktop/LLMs/Mixtral4bit'
generator_config.prepare()

generator_model = ExLlamaV2(generator_config)
cache = ExLlamaV2Cache(generator_model, lazy = True)

print("Loading model...")
generator_model.load_autosplit(cache)

generator_tokenizer = ExLlamaV2Tokenizer(generator_config)
generator_llm = ExLlamaV2StreamingGenerator(generator_model, cache, generator_tokenizer)
generator_llm.set_stop_conditions([generator_tokenizer.eos_token_id])
generator_settings = ExLlamaV2Sampler.Settings()
generator_settings.temperature = 0.85
generator_settings.top_k = 50
generator_settings.top_p = 0.8
generator_settings.token_repetition_penalty = 1.01
#generator_settings.disallow_tokens(generator_tokenizer, [generator_tokenizer.eos_token_id])
# see if commenting out the above solved the endless generation issue (did not have with stream generator)

Loading model...


In [69]:
#help(generator_llm)

In [70]:
#help(generator_tokenizer)

In [71]:
#help(generator_llm)

In [72]:
#help(generator_model)

In [73]:
# from transformers import Pipeline
# from ragatouille import RAGPretrainedModel
# from typing import Optional, List, Tuple
# from langchain.docstore.document import Document
# import time
# # RAG_PROMPT_TEMPLATE = tokenizer.apply_chat_template(

# #     prompt_in_chat_format, tokenize=False, add_generation_prompt=True

# # )
# RERANKER = RAGPretrainedModel.from_pretrained("colbert-ir/colbertv2.0")
# from langchain.docstore.document import Document as LangchainDocument
# def call_llm(
#     question: str,
#     generator_model: ExLlamaV2,
#     generator_llm: ExLlamaV2StreamingGenerator,
#     tokenizer: ExLlamaV2Tokenizer,
#     settings:ExLlamaV2Sampler.Settings,
#     max_new_tokens = 512
# ) -> Tuple[str, List[LangchainDocument]]:


#     instruction_ids = tokenizer.encode(f"<s>[INST] {question} [/INST]", add_bos = True)
#     context_ids = instruction_ids if generator_llm.sequence_ids is None \
#             else torch.cat([generator_llm.sequence_ids, instruction_ids], dim = -1)

#     max_new_tokens = max_new_tokens

#     generator_llm.warmup()
#     time_begin = time.time()

#     output = generator_model.forward(f"<s>[INST] {question} [/INST]", settings, max_new_tokens)#, seed = 1234)#,add_eos=True)
    
#     time_end = time.time()
#     time_total = time_end - time_begin

#     print(output)
#     print()
#     print(f"Response generated in {time_total:.2f} seconds, {max_new_tokens} tokens, {max_new_tokens / time_total:.2f} tokens/second")
#     return output
# #     generator.begin_stream(context_ids, settings)

# # #    return generator.generate_simple(context_ids, settings,num_tokens=512)

# #     while True:
# #         chunk, eos, _ = generator.stream()
# #         if eos: break
# #         print(chunk, end = "")
# #         sys.stdout.flush()
# #     #####




# # def call_llm(inference_client: InferenceClient, prompt: str):
# #     response = inference_client.post(
# #         json={
# #             "inputs": prompt,
# #             "parameters": {"max_new_tokens": 1000},
# #             "task": "text-generation",
# #         },
# #     )
# #     return json.loads(response.decode())[0]["generated_text"]


# call_llm(question="How can I get my cat to like me?", generator_model=generator_model,generator_llm=generator_llm,tokenizer=generator_tokenizer,settings=generator_settings)

In [13]:
#Working except eos
from transformers import Pipeline
from ragatouille import RAGPretrainedModel
from typing import Optional, List, Tuple
from langchain.docstore.document import Document
import time
# RAG_PROMPT_TEMPLATE = tokenizer.apply_chat_template(

#     prompt_in_chat_format, tokenize=False, add_generation_prompt=True

# )
RERANKER = RAGPretrainedModel.from_pretrained("colbert-ir/colbertv2.0")
from langchain.docstore.document import Document as LangchainDocument
def call_llm(
    question: str,
    generator: ExLlamaV2StreamingGenerator,
    tokenizer: ExLlamaV2Tokenizer,
    settings:ExLlamaV2Sampler.Settings,
    max_new_tokens = 512
) -> Tuple[str, List[LangchainDocument]]:


    # instruction_ids = tokenizer.encode(f"<s>[INST] {question} [/INST]", add_bos = True)
    # context_ids = instruction_ids if generator.sequence_ids is None \
    #         else torch.cat([generator.sequence_ids, instruction_ids], dim = -1)

    max_new_tokens = max_new_tokens

    generator.warmup()
    #time_begin = time.time()
    output = generator.generate_simple(f"<s>[INST] {question} [/INST]", settings, max_new_tokens, seed = 1234)

    #output = generator.generate_simple(f"<s>[INST] {question} [/INST]", settings, max_new_tokens, seed = 1234)
    #output = generator.generate_simple(question, settings, max_new_tokens, seed = 1234)

    # time_end = time.time()
    # time_total = time_end - time_begin

    #print(output)
    #print()
    #print(f"Response generated in {time_total:.2f} seconds, {max_new_tokens} tokens, {max_new_tokens / time_total:.2f} tokens/second")
    return output
#     generator.begin_stream(context_ids, settings)

# #    return generator.generate_simple(context_ids, settings,num_tokens=512)

#     while True:
#         chunk, eos, _ = generator.stream()
#         if eos: break
#         print(chunk, end = "")
#         sys.stdout.flush()
#     #####




# def call_llm(inference_client: InferenceClient, prompt: str):
#     response = inference_client.post(
#         json={
#             "inputs": prompt,
#             "parameters": {"max_new_tokens": 1000},
#             "task": "text-generation",
#         },
#     )
#     return json.loads(response.decode())[0]["generated_text"]


call_llm(question="How can I get my cat to like me?", generator=generator_llm,tokenizer=generator_tokenizer,settings=generator_settings,max_new_tokens=1024)

'<s>[INST] How can I get my cat to like me? [/INST] 1. Spend time with your cat: Cats enjoy spending time with their owners, so take some time out of your day to play with them or simply sit with them.\n\n2. Provide food and shelter: Ensure that your cat has access to good food and a comfortable place to sleep.\n\n3. Show affection: Cats love physical touch, so try petting them or giving them a gentle scratch behind their ears.\n\n4. Play with toys: Cats enjoy playing with toys, so try introducing them to some new ones.\n\n5. Be patient: Cats can be slow to warm up to new people, so be patient and give them time to get used to you.\n\n6. Use positive reinforcement: Reward your cat with treats or praise when they interact with you positively.\n\n7. Avoid loud noises: Cats can be easily startled by loud noises, so try to keep your voice and movements calm around them.\n\n8. Be consistent: Consistency is key when it comes to building a relationship with your cat. Try to spend the same amo

In [75]:
# QA_generation_prompt = """
# Your task is to write a factoid question and an answer given a context.
# Your factoid question should be answerable with a specific, concise piece of factual information from the context.
# Your factoid question should be formulated in the same style as questions users could ask in a search engine.
# This means that your factoid question MUST NOT mention something like "according to the passage" or "context".

# Provide your answer as follows:

# Output:::
# Factoid question: (your factoid question)
# Answer: (your answer to the factoid question)

# Now here is the context.

# Context: {context}\n
# Output:::"""

In [14]:
QA_generation_prompt = """
Your task is to write a deep factual or conceptual question and an answer given a context.
Your deep question should be unambigiously answerable from the context.
Your deep question should be formulated in the same style as questions people reading advanced LLM papers would ask.
This means that your question MUST NOT mention something like "according to the passage" or "context".

Provide your answer as follows:

Output:::
Deep question: (your deep question)
Answer: (your answer to the deep question)

Now here is the context.

Context: {context}\n
Output:::"""

Now let's generate our QA couples.
For this example, we generate only 10 QA couples and will load the rest from the Hub.

But for your specific knowledge base, given that you want to get at least ~100 test samples, and accounting for the fact that we will filter out around half of these with our critique agents later on, you should generate much more, in the >200 samples.

In [15]:
import random
from tqdm import tqdm
N_GENERATIONS = 300  # We intentionally generate only 10 QA couples here for cost and time considerations

print(f"Generating {N_GENERATIONS} QA couples...")

outputs = []
for sampled_context in tqdm(random.sample(docs_processed, N_GENERATIONS)):
    # Generate QA couple
    # output_QA_couple = call_llm(
    #     llm_client, QA_generation_prompt.format(context=sampled_context.page_content)
    # )
    output_QA_couple = call_llm(question=QA_generation_prompt.format(context=sampled_context.page_content), generator=generator_llm,tokenizer=generator_tokenizer,settings=generator_settings,
                                max_new_tokens=1024)
    try:
        question = output_QA_couple.split("Deep question: ")[-1].split("Answer: ")[0]
        answer = output_QA_couple.split("Answer: ")[-1]
        #assert len(answer) < 300, "Answer is too long"
        outputs.append(
            {
                "context": sampled_context.page_content,
                "question": question,
                "answer": answer,
                "source_doc": sampled_context.metadata["source"],
            }
        )
    except:
        continue

Generating 300 QA couples...


100%|██████████| 300/300 [07:15<00:00,  1.45s/it]


In [18]:
outputs

[{'context': 'Skeleton-of-Thought: Large Language Models Can Do Parallel Decoding\nSKELETON -OF-THOUGHT : L ARGE LANGUAGE MOD-\nELSCANDOPARALLEL DECODING\nXuefei Ning1∗\nfoxdoraame@gmail.comZinan Lin2∗\nlinzinan1995@gmail.com\nZixuan Zhou1∗\nzhouzx21@mails.tsinghua.edu.cnZifu Wang3\nzifu.wang@kuleuven.be\nHuazhong Yang1\nyanghz@tsinghua.edu.cnYu Wang1\nyu-wang@tsinghua.edu.cn\n1Department of Electronic Engineering, Tsinghua University, Beijing, China\n2Microsoft Research, Redmond, Washington, USA\n3ESAT-PSI, KU Leuven, Leuven, Belgium\nWebsite: https://sites.google.com/view/sot-llm\nABSTRACT\nThis work aims at decreasing the end-to-end generation latency of large language\nmodels (LLMs). One of the major causes of the high generation latency is the\nsequential decoding approach adopted by almost all state-of-the-art LLMs. In\nthis work, motivated by the thinking and writing process of humans, we propose\nSkeleton-of-Thought (SoT) , which first guides LLMs to generate the skeleton of\nt

In [20]:
import pandas as pd
pd.set_option('display.max_colwidth',800)
display(pd.DataFrame(outputs).head(10))

### 1.3. Setup critique agents

The questions generated by the previous agent can have many flaws: we should do a quality check before validating these questions.

We thus build critique agents that will rate each question on several criteria, given in [this paper](https://huggingface.co/papers/2312.10003):
- **Groundedness:** can the question be answered from the given context?
- **Relevance:** is the question relevant to users? For instance, `"What is the date when transformers 4.29.1 was released?"` is not relevant for ML practicioners.

One last failure case we've noticed is when a function is tailored for the particular setting where the question was generated, but undecipherable by itself, like `"What is the name of the function used in this guide?"`.
We also build a critique agent for this criteria:
- **Stand-alone**: is the question understandable free of any context, for someone with domain knowledge/Internet access? The opposite of this would be `What is the function used in this article?` for a question generated from a specific blog article.

We systematically score functions with all these agents, and whenever the score is too low for any one of the agents, we eliminate the question from our eval dataset.

💡 ___When asking the agents to output a score, we first ask them to produce its rationale. This will help us verify scores, but most importantly, asking it to first output rationale gives the model more tokens to think and elaborate an answer before summarizing it into a single score token.___

We now build and run these critique agents.

In [ ]:


### Semi-working backup
# question_groundedness_critique_prompt = """
# You will be given a context and a question.
# Your task is to provide a 'total rating' scoring how well one can answer the given question unambiguously with the given context.
# Give your answer on a scale of 1 to 5, where 1 means that the question is not answerable at all given the context, and 5 means that the question is clearly and unambiguously answerable with the context.

# Provide your answer as follows:

# Answer:::
# Evaluation: (your rationale for the rating, as a text)
# Total rating: (your rating, as a number between 1 and 5)

# You MUST provide values for 'Evaluation:' and 'Total rating:' in your answer.

# Now here are the question and context.

# Question: {question}\n
# Context: {context}\n
# Answer::: """

# question_relevance_critique_prompt = """
# You will be given a question.
# Your task is to provide a 'total rating' representing how useful this question can be to machine learning developers building NLP applications with the Hugging Face ecosystem.
# Give your answer on a scale of 1 to 5, where 1 means that the question is not useful at all, and 5 means that the question is extremely useful.

# Provide your answer as follows:

# Answer:::
# Evaluation: (your rationale for the rating, as a text)
# Total rating: (your rating, as a number between 1 and 5)

# You MUST provide values for 'Evaluation:' and 'Total rating:' in your answer.

# Now here is the question.

# Question: {question}\n
# Answer::: """

# question_standalone_critique_prompt = """
# You will be given a question.
# Your task is to provide a 'total rating' representing how context-independent this question is.
# Give your answer on a scale of 1 to 5, where 1 means that the question depends on additional information to be understood, and 5 means that the question makes sense by itself.
# For instance, if the question refers to a particular setting, like 'in the context' or 'in the document', the rating must be 1.
# The questions can contain obscure technical nouns or acronyms like Gradio, Hub, Hugging Face or Space and still be a 5: it must simply be clear to an operator with access to documentation what the question is about.

# For instance, "What is the name of the checkpoint from which the ViT model is imported?" should receive a 1, since there is an implicit mention of a context, thus the question is not independent from the context.

# Provide your answer as follows:

# Answer:::
# Evaluation: (your rationale for the rating, as a text)
# Total rating: (your rating, as a number between 1 and 5)

# You MUST provide values for 'Evaluation:' and 'Total rating:' in your answer.

# Now here is the question.

# Question: {question}\n
# Answer::: """

In [21]:

## Semi-working backup
question_groundedness_critique_prompt = """
You will be given a context and a question.
Your task is to provide a 'total rating' scoring how well one can answer the given question unambiguously with the given context.
Give your answer on a scale of 1 to 5, where 1 means that the question is not answerable at all given the context, and 5 means that the question is clearly and unambiguously answerable with the context.

Provide your answer as follows:

Answer:::
Evaluation: (your rationale for the rating, as a text)
Total rating: !!!(your rating, as a number between 1 and 5)!!!

You MUST provide values for 'Evaluation:' and 'Total rating:' in your answer.  'Total rating:' should be encolsed in !!! and !!! as in '!!!4.5!!!'.


Now here are the question and context.

Question: {question}\n
Context: {context}\n
Answer::: """

question_relevance_critique_prompt = """
You will be given a question.
Your task is to provide a 'total rating' representing how useful this question can be to machine learning developers building NLP applications with the Hugging Face ecosystem.
Give your answer on a scale of 1 to 5, where 1 means that the question is not useful at all, and 5 means that the question is extremely useful.

Provide your answer as follows:

Answer:::
Evaluation: (your rationale for the rating, as a text)
Total rating: !!!(your rating, as a number between 1 and 5)!!!

You MUST provide values for 'Evaluation:' and 'Total rating:' in your answer.  'Total rating:' should be encolsed in !!! and !!! as in '!!!4.5!!!'.


Now here is the question.

Question: {question}\n
Answer::: """

question_standalone_critique_prompt = """
You will be given a question.
Your task is to provide a 'total rating' representing how context-independent this question is.
Give your answer on a scale of 1 to 5, where 1 means that the question depends on additional information to be understood, and 5 means that the question makes sense by itself.
For instance, if the question refers to a particular setting, like 'in the context' or 'in the document', the rating must be 1.
The questions can contain obscure technical nouns or acronyms like Gradio, Hub, Hugging Face or Space and still be a 5: it must simply be clear to an operator with access to documentation what the question is about.

For instance, "What is the name of the checkpoint from which the ViT model is imported?" should receive a 1, since there is an implicit mention of a context, thus the question is not independent from the context.

Provide your answer as follows:

Answer:::
Evaluation: (your rationale for the rating, as a text)
Total rating: !!!(your rating, as a number between 1 and 5)!!!

You MUST provide values for 'Evaluation:' and 'Total rating:' in your answer.  'Total rating:' should be encolsed in !!! and !!! as in '!!!4.5!!!'.


Now here is the question.

Question: {question}\n
Answer::: """

In [ ]:


# ## Semi-working backup
# question_groundedness_critique_prompt = """
# You will be given a context and a question.
# Your task is to provide a 'total rating' scoring how well one can answer the given question unambiguously with the given context.
# Give your answer on a scale of 1 to 5, where 1 means that the question is not answerable at all given the context, and 5 means that the question is clearly and unambiguously answerable with the context.

# Provide your answer as follows:

# Answer:::
# Evaluation: (your rationale for the rating, as a text)
# Total rating: !!!(your rating, as a number between 1 and 5)!!!

# You MUST provide values for 'Evaluation:' and 'Total rating:' in your answer.  'Total rating:' should be preceded and followd by '!!!' as in 'Total rating: !!!4.5!!!'

# Now here are the question and context.

# Question: {question}\n
# Context: {context}\n
# Answer::: """

# question_relevance_critique_prompt = """
# You will be given a question.
# Your task is to provide a 'total rating' representing how useful this question can be to machine learning developers building NLP applications with the Hugging Face ecosystem.
# Give your answer on a scale of 1 to 5, where 1 means that the question is not useful at all, and 5 means that the question is extremely useful.

# Provide your answer as follows:

# Answer:::
# Evaluation: (your rationale for the rating, as a text)
# Total rating: !!!(your rating, as a number between 1 and 5)!!!

# You MUST provide values for 'Evaluation:' and 'Total rating:' in your answer.  'Total rating:' should be preceded and followd by '!!!' as in 'Total rating: !!!4.5!!!'

# Now here is the question.

# Question: {question}\n
# Answer::: """

# question_standalone_critique_prompt = """
# You will be given a question.
# Your task is to provide a 'total rating' representing how context-independent this question is.
# Give your answer on a scale of 1 to 5, where 1 means that the question depends on additional information to be understood, and 5 means that the question makes sense by itself.
# For instance, if the question refers to a particular setting, like 'in the context' or 'in the document', the rating must be 1.
# The questions can contain obscure technical nouns or acronyms like Gradio, Hub, Hugging Face or Space and still be a 5: it must simply be clear to an operator with access to documentation what the question is about.

# For instance, "What is the name of the checkpoint from which the ViT model is imported?" should receive a 1, since there is an implicit mention of a context, thus the question is not independent from the context.

# Provide your answer as follows:

# Answer:::
# Evaluation: (your rationale for the rating, as a text)
# Total rating: !!!(your rating, as a number between 1 and 5)!!!

# You MUST provide values for 'Evaluation:' and 'Total rating:' in your answer.  'Total rating:' should be preceded and followd by '!!!' as in 'Total rating: !!!4.5!!!'

# Now here is the question.

# Question: {question}\n
# Answer::: """

In [22]:
# import re
# print("Generating critique for each QA couple...")
# for output in tqdm(outputs[:1]):
#     evaluations = {
#         "groundedness": call_llm(question=question_groundedness_critique_prompt.format(context=output["context"], question=output["question"]), 
#                                 generator=generator_llm,
#                                 tokenizer=generator_tokenizer,settings=generator_settings,
#                                 max_new_tokens=1024),
#         "relevance": call_llm(question=question_relevance_critique_prompt.format(question=output["question"]), 
#                                 generator=generator_llm,
#                                 tokenizer=generator_tokenizer,settings=generator_settings,
#                                 max_new_tokens=1024),
                    
#         "standalone": call_llm(question=question_standalone_critique_prompt.format(question=output["question"]),
#                                 generator=generator_llm,
#                                 tokenizer=generator_tokenizer,settings=generator_settings,
#                                 max_new_tokens=1024)
#     }
#     try:
#         # for criterion, evaluation in evaluations.items():
#         #     # score, eval = (
#         #     #     (evaluation.split("Total rating: ")[-1].strip()),
#         #     #     evaluation.split("Total rating: ")[-2].split("Evaluation: ")[1],
#         #     # )
#         #     score, eval = (
#         #         float(evaluation.split("!!!")[1]),
#         #         evaluation.split("Total rating: ")[-2].split("Evaluation: ")[1],
#         #     )
#         #     output.update(
#         #         {
#         #             f"{criterion}_score": score,
#         #             f"{criterion}_eval": eval,
#         #         }
#         #     )
#         for criterion, evaluation in evaluations.items():
#             score_match = re.search(r'Total rating: !!!([\d\.]+)!!!', evaluation)
#             #re.search(r'Total rating: !!!([\d\.]+)!!!.*', evaluation)
#             eval_match = re.search(r'Evaluation: (.*?)Total rating:', evaluation)
            
#             if score_match and eval_match:
#                 score = float(score_match.group(1))
#                 eval = eval_match.group(1).strip()
#                 output.update(
#                     {
#                         f"{criterion}_score": score,
#                         f"{criterion}_eval": eval,
#                     }
#                 )
#     except Exception as e:
#         print("\033[91m" + f"EVALUATION:" + "\033[0m")
#         print(evaluations)
#         print("\033[91m" + f"EXCEPTION: {e}" + "\033[0m")
#         break
#         #continue

In [23]:
question_groundedness_critique_prompt = """
You will be given a context and a question.
Your task is to provide a 'total rating' scoring how well one can answer the given question unambiguously with the given context.
Give your answer on a scale of 1 to 5, where 1 means that the question is not answerable at all given the context, and 5 means that the question is clearly and unambiguously answerable with the context.

Provide your answer as follows:

Answer:::
Evaluation: (your rationale for the rating, as a text)
Total rating: (your rating, as a number between 1 and 5)

You MUST provide values for 'Evaluation:' and 'Total rating:' in your answer.

Now here are the question and context.

Question: {question}\n
Context: {context}\n
Answer::: """

question_relevance_critique_prompt = """
You will be given a question.
Your task is to provide a 'total rating' representing how useful this question can be to machine learning developers building NLP applications with the Hugging Face ecosystem.
Give your answer on a scale of 1 to 5, where 1 means that the question is not useful at all, and 5 means that the question is extremely useful.

Provide your answer as follows:

Answer:::
Evaluation: (your rationale for the rating, as a text)
Total rating: (your rating, as a number between 1 and 5)

You MUST provide values for 'Evaluation:' and 'Total rating:' in your answer.

Now here is the question.

Question: {question}\n
Answer::: """

question_standalone_critique_prompt = """
You will be given a question.
Your task is to provide a 'total rating' representing how context-independant this question is.
Give your answer on a scale of 1 to 5, where 1 means that the question depends on additional information to be understood, and 5 means that the question makes sense by itself.
For instance, if the question refers to a particular setting, like 'in the context' or 'in the document', the rating must be 1.
The questions can contain obscure technical nouns or acronyms like Gradio, Hub, Hugging Face or Space and still be a 5: it must simply be clear to an operator with access to documentation what the question is about.

For instance, "What is the name of the checkpoint from which the ViT model is imported?" should receive a 1, since there is an implicit mention of a context, thus the question is not independant from the context.

Provide your answer as follows:

Answer:::
Evaluation: (your rationale for the rating, as a text)
Total rating: (your rating, as a number between 1 and 5)

You MUST provide values for 'Evaluation:' and 'Total rating:' in your answer.

Now here is the question.

Question: {question}\n
Answer::: """

In [50]:
print("Generating critique for each QA couple...")
for output in tqdm(outputs):
    evaluations = {
        "groundedness": call_llm(question=question_groundedness_critique_prompt.format(context=output["context"], question=output["question"]), 
                                generator=generator_llm,
                                tokenizer=generator_tokenizer,settings=generator_settings,
                                max_new_tokens=1024),
        "relevance": call_llm(question=question_relevance_critique_prompt.format(question=output["question"]), 
                                generator=generator_llm,
                                tokenizer=generator_tokenizer,settings=generator_settings,
                                max_new_tokens=1024),
                    
        "standalone": call_llm(question=question_standalone_critique_prompt.format(question=output["question"]),
                                generator=generator_llm,
                                tokenizer=generator_tokenizer,settings=generator_settings,
                                max_new_tokens=1024)
    }
    try:
        for criterion, evaluation in evaluations.items():
            score, eval = (
                # int(evaluation.split("Total rating: ")[-1].strip()),
                (evaluation.split("Total rating: ")[-1].strip()),
                evaluation.split("Total rating: ")[-2].split("Evaluation: ")[1],
            )
            output.update(
                {
                    f"{criterion}_score": score,
                    f"{criterion}_eval": eval,
                }
            )
    except Exception as e:
        #print("\033[91m" + f"EVALUATION:" + "\033[0m")
        #print(evaluations)
        #print("\033[91m" + f"EXCEPTION: {e}" + "\033[0m")
        continue

In [25]:
evaluations

{'groundedness': "<s>[INST] \nYou will be given a context and a question.\nYour task is to provide a 'total rating' scoring how well one can answer the given question unambiguously with the given context.\nGive your answer on a scale of 1 to 5, where 1 means that the question is not answerable at all given the context, and 5 means that the question is clearly and unambiguously answerable with the context.\n\nProvide your answer as follows:\n\nAnswer:::\nEvaluation: (your rationale for the rating, as a text)\nTotal rating: (your rating, as a number between 1 and 5)\n\nYou MUST provide values for 'Evaluation:' and 'Total rating:' in your answer.\n\nNow here are the question and context.\n\nQuestion: Can incorporating ReAct and CoT-SC improve the problem solving process in reasoning structure?\n\n\n\nContext: and reinforcement learning instead of prompting.\nCombining Internal and External Knowledge As will be detail in Section 3.3, we observe that\nthe problem solving process demonstrate

Now let us filter out bad questions based on our critique agent scores:

In [26]:
outputs[0].keys()

dict_keys(['context', 'question', 'answer', 'source_doc', 'groundedness_score', 'groundedness_eval'])

In [28]:
# import pandas as pd

# pd.set_option("display.max_colwidth", None)

# generated_questions = pd.DataFrame.from_dict(output)

# print("Evaluation dataset before filtering:")
# display(
#     generated_questions.reindex(
#         [
#             "question",
#             "answer",
#             "groundedness_score",
#             "relevance_score",
#             "standalone_score",
#         ],
#         axis=1)
    
# )

In [38]:
import pandas as pd

pd.set_option("display.max_colwidth", None)

generated_questions = pd.DataFrame.from_dict(outputs)

print("Evaluation dataset before filtering:")
display(
    generated_questions[
        [
            "question",
            "answer",
            "groundedness_score",
            "relevance_score",
            "standalone_score",
        ]
    ]
)

Evaluation dataset before filtering:


,question,answer,groundedness_score,relevance_score,standalone_score
0,Can a large language model with parallel decoding be designed to think more like a human for answer quality?\n\n,"Yes, the proposed Skeleton-of-Thought (SoT) method in the context is an attempt to optimize the inference efficiency of large language models (LLMs) for data-centric optimization, which can potentially improve the answer quality on several question categories. This is achieved by guiding the LLMs to generate the skeleton of the answer first and then conducting parallel API calls or batched decoding to complete the contents of each skeleton point in parallel. The method is an initial attempt at data-centric optimization for inference efficiency and underscores the potential of pushing LLMs to think more like a human for answer quality.","3\n\nTherefore, the total rating is 3 out of 5.",NaN,NaN
1,"Does the dataset augmentation with samples from the model improve the performance of the scratchpad model, while it harms the direct execution model?\n\n","Yes, the dataset augmentation with samples from the model improves the performance of the scratchpad model, while it harms the direct execution model.",3,NaN,NaN
2,"In relation to the results presented in Table 3, what is the significance of the differences in word-level success rates between IO, CoT, Oracle, and ToT prompting methods, and how do these differences impact the performance of the models in solving crossword games?\n\n","The results in Table 3 show that IO and CoT prompting methods have a word-level success rate of less than 16%, while ToT significantly improves all metrics, achieving a word-level success rate of 60% and solving 4 out of 20 games. This indicates that ToT has mechanisms in place to try different clues, make changes to decisions, and backtrack, which are lacking in IO and CoT. Additionally, when outputting from the oracle best DFS state, ToT performance is even higher and actually solves 7/20 games, indicating that the simple output heuristics used can be improved. The importance of backtracking is also confirmed by an ablation that keeps filling the most promising clue for at most 20 steps, which performs poorly with a word-level success rate of only 20%.",NaN,NaN,NaN
3,"Given the context, how does the Mini Crosswords algorithm propose and aggregate thoughts in a priority queue during a depth-first search (DFS) for solving crossword puzzles?\n\n","The Mini Crosswords algorithm proposes and aggregates thoughts in a priority queue during a depth-first search (DFS) for solving crossword puzzles by first generating all possible thoughts for each given clue. These thoughts are then evaluated based on the possibility of filling in each remaining word clue, and pruned if any remaining clue is deemed not possible to fill by the LM. The remaining thoughts are then sorted in a priority queue based on their evaluation scores, and the algorithm explores the next promising thought for clue in the priority queue. This process continues until the deepest explored state is rendered into the final output.",4.5,"4\n\nThe Mini Crosswords algorithm is a well-documented and widely used method for solving crossword puzzles, and its DFS approach is a common technique in NLP applications. Additionally, the use of a priority queue to aggregate thoughts is a common approach in NLP, and understanding how this process works can be useful for developers building NLP applications. However, the specifics of the Mini Crosswords algorithm may not be applicable to all NLP applications, and the question could benefit from more context and specificity.",5
4,How can dialogue models be improved to better align with human judgements?\n\n,"Dialogue models can be improved to better align with human judgements by incorporating targeted human judgements into the training process. This can be done through methods such as active learning and human-in-the-loop approaches. Additionally, improving the quality of the tra

In [39]:
generated_questions['groundedness_score']=generated_questions['groundedness_score'].astype(str).str.extract(r'(\d+\.?\d*)').astype(float)
generated_questions['relevance_score']=generated_questions['relevance_score'].astype(str).str.extract(r'(\d+\.?\d*)').astype(float)
generated_questions['standalone_score']=generated_questions['groundedness_score'].astype(str).str.extract(r'(\d+\.?\d*)').astype(float)

In [34]:
generated_questions.to_csv("./data/generated_questions_raw.csv", index=False)

In [40]:
display(
    generated_questions[
        [
            "question",
            "answer",
            "groundedness_score",
            "relevance_score",
            "standalone_score",
        ]
    ]
)

,question,answer,groundedness_score,relevance_score,standalone_score
0,Can a large language model with parallel decoding be designed to think more like a human for answer quality?\n\n,"Yes, the proposed Skeleton-of-Thought (SoT) method in the context is an attempt to optimize the inference efficiency of large language models (LLMs) for data-centric optimization, which can potentially improve the answer quality on several question categories. This is achieved by guiding the LLMs to generate the skeleton of the answer first and then conducting parallel API calls or batched decoding to complete the contents of each skeleton point in parallel. The method is an initial attempt at data-centric optimization for inference efficiency and underscores the potential of pushing LLMs to think more like a human for answer quality.",3.0,NaN,3.0
1,"Does the dataset augmentation with samples from the model improve the performance of the scratchpad model, while it harms the direct execution model?\n\n","Yes, the dataset augmentation with samples from the model improves the performance of the scratchpad model, while it harms the direct execution model.",3.0,NaN,3.0
2,"In relation to the results presented in Table 3, what is the significance of the differences in word-level success rates between IO, CoT, Oracle, and ToT prompting methods, and how do these differences impact the performance of the models in solving crossword games?\n\n","The results in Table 3 show that IO and CoT prompting methods have a word-level success rate of less than 16%, while ToT significantly improves all metrics, achieving a word-level success rate of 60% and solving 4 out of 20 games. This indicates that ToT has mechanisms in place to try different clues, make changes to decisions, and backtrack, which are lacking in IO and CoT. Additionally, when outputting from the oracle best DFS state, ToT performance is even higher and actually solves 7/20 games, indicating that the simple output heuristics used can be improved. The importance of backtracking is also confirmed by an ablation that keeps filling the most promising clue for at most 20 steps, which performs poorly with a word-level success rate of only 20%.",NaN,NaN,NaN
3,"Given the context, how does the Mini Crosswords algorithm propose and aggregate thoughts in a priority queue during a depth-first search (DFS) for solving crossword puzzles?\n\n","The Mini Crosswords algorithm proposes and aggregates thoughts in a priority queue during a depth-first search (DFS) for solving crossword puzzles by first generating all possible thoughts for each given clue. These thoughts are then evaluated based on the possibility of filling in each remaining word clue, and pruned if any remaining clue is deemed not possible to fill by the LM. The remaining thoughts are then sorted in a priority queue based on their evaluation scores, and the algorithm explores the next promising thought for clue in the priority queue. This process continues until the deepest explored state is rendered into the final output.",4.5,4.0,4.5
4,How can dialogue models be improved to better align with human judgements?\n\n,"Dialogue models can be improved to better align with human judgements by incorporating targeted human judgements into the training process. This can be done through methods such as active learning and human-in-the-loop approaches. Additionally, improving the quality of the training data and using more advanced techniques such as transfer learning and low-rank hypercomplex adapter layers can also help improve alignment.",4.5,4.0,4.5
...,...,...,...,...,...
295,What is the difference between traditional prompting and chain-of-thought prompting in language models?\n\n,"Chain-of-thought prompting involves providing demonstrations of intermediate reasoning steps to language models, which allows them to generate a coherent series of steps leading to the final answer for a problem. This approach contrasts with traditional prompting, which does n

In [41]:
for col in ["groundedness_score", "relevance_score", "standalone_score"]:
    generated_questions[col] = generated_questions[col].fillna(generated_questions[["groundedness_score", "relevance_score", "standalone_score"]].min(axis=1))

In [42]:

generated_questions = generated_questions.loc[
    (generated_questions["groundedness_score"] >= 3.0)
    & (generated_questions["relevance_score"] >= 3.0)
    & (generated_questions["standalone_score"] >= 3.0)
]
print("============================================")
print("Final evaluation dataset:")
display(
    generated_questions[
        [
            "question",
            "answer",
            "groundedness_score",
            "relevance_score",
            "standalone_score",
        ]
    ]
)

# eval_dataset = datasets.Dataset.from_pandas(
#     generated_questions, split="train", preserve_index=False
# )

Final evaluation dataset:


,question,answer,groundedness_score,relevance_score,standalone_score
0,Can a large language model with parallel decoding be designed to think more like a human for answer quality?\n\n,"Yes, the proposed Skeleton-of-Thought (SoT) method in the context is an attempt to optimize the inference efficiency of large language models (LLMs) for data-centric optimization, which can potentially improve the answer quality on several question categories. This is achieved by guiding the LLMs to generate the skeleton of the answer first and then conducting parallel API calls or batched decoding to complete the contents of each skeleton point in parallel. The method is an initial attempt at data-centric optimization for inference efficiency and underscores the potential of pushing LLMs to think more like a human for answer quality.",3.0,3.0,3.0
1,"Does the dataset augmentation with samples from the model improve the performance of the scratchpad model, while it harms the direct execution model?\n\n","Yes, the dataset augmentation with samples from the model improves the performance of the scratchpad model, while it harms the direct execution model.",3.0,3.0,3.0
3,"Given the context, how does the Mini Crosswords algorithm propose and aggregate thoughts in a priority queue during a depth-first search (DFS) for solving crossword puzzles?\n\n","The Mini Crosswords algorithm proposes and aggregates thoughts in a priority queue during a depth-first search (DFS) for solving crossword puzzles by first generating all possible thoughts for each given clue. These thoughts are then evaluated based on the possibility of filling in each remaining word clue, and pruned if any remaining clue is deemed not possible to fill by the LM. The remaining thoughts are then sorted in a priority queue based on their evaluation scores, and the algorithm explores the next promising thought for clue in the priority queue. This process continues until the deepest explored state is rendered into the final output.",4.5,4.0,4.5
4,How can dialogue models be improved to better align with human judgements?\n\n,"Dialogue models can be improved to better align with human judgements by incorporating targeted human judgements into the training process. This can be done through methods such as active learning and human-in-the-loop approaches. Additionally, improving the quality of the training data and using more advanced techniques such as transfer learning and low-rank hypercomplex adapter layers can also help improve alignment.",4.5,4.0,4.5
5,"In the context of TeacherLM-7.1B, what are the characteristics and deficiencies of the generation content compared to human annotation and text-davinci-003?\n\n","TeacherLM-7.1B's explanations are generally more comprehensive and detailed than human annotations and text-davinci-003's explanations. However, it falls behind text-davinci-003 in solving mathematical problems, probably related to the model's size.",3.5,4.0,3.5
...,...,...,...,...,...
288,Can explicit articulation of the thought process in language elicit high-quality answers from LLMs?\n\n,"Yes, recent research has shown that explicitly articulating the thought process in language can elicit high-quality answers from LLMs. Examples include CoT (Kojima et al., 2022; Wei et al., 2022), ToT (Yao et al., 2023), and ReAct (Yao et al., 2022). These findings resemble human thinking and suggest that LLMs can benefit from drawing from the human thinking process to facilitate more effective and efficient AI.",4.0,4.0,4.0
291,What is the difference between GPT-12B and Codex in terms of their performance on the HumanEval dataset?\n\n,"Codex outperforms GPT-12B in generating standalone Python functions from docstrings on the HumanEval dataset. When a single sample is generated for each problem, Codex solves 28.8% of the problems, while GPT-12B solves no problems. However, when generating 100 samples per problem and selecting the sample with the highest mean log-probability, Codex solves 44.5%

In [43]:
generated_questions.to_csv("./data/generated_questions_filtered.csv", index=False)

In [51]:
#generated_questions.head()

- Go through the 181 rows remaining post dropping missing vals and missing value imputation visually, keep the better 120ish questions
    - Dropped questions that were off-target for learning about LLMs, relied on the reference section, or mentioned the context.

In [46]:
eval_dataset = pd.read_csv("../data/generated_questions_filtered_w_human.csv")

In [47]:
eval_dataset.shape

(121, 10)

In [3]:
#eval_dataset.head()

# 2. Build our RAG System

### 2.1. Preprocessing documents to build our vector database

- In this part, __we split the documents from our knowledge base into smaller chunks__: these will be the snippets that are picked by the Retriever, to then be ingested by the Reader LLM as supporting elements for its answer.
- The goal is to build semantically relevant snippets: not too small to be sufficient for supporting an answer, and not too large too avoid diluting individual ideas.

Many options exist for text splitting:
- split every `n` words / characters, but this has the risk of cutting in half paragraphs or even sentences
- split after `n` words / character, but only on sentence boundaries
- **recursive split** tries to preserve even more of the document structure, by processing it tree-like way, splitting first on the largest units (chapters) then recursively splitting on smaller units (paragraphs, sentences).

To learn more about chunking, I recommend you read [this great notebook](https://github.com/FullStackRetrieval-com/RetrievalTutorials/blob/main/5_Levels_Of_Text_Splitting.ipynb) by Greg Kamradt.

[This space](https://huggingface.co/spaces/m-ric/chunk_visualizer) lets you visualize how different splitting options affect the chunks you get.

> In the following, we use Langchain's `RecursiveCharacterTextSplitter`.

💡 _To measure chunk length in our Text Splitter, our length function will not be the count of characters, but the count of tokens in the tokenized text: indeed, for subsequent embedder that processes token, measuring length in tokens is more relevant and empirically performs better._

In [58]:
# from langchain.docstore.document import Document as LangchainDocument

# RAW_KNOWLEDGE_BASE = [
#     LangchainDocument(page_content=doc["text"], metadata={"source": doc["source"]})
#     for doc in tqdm(eval_dataset)
# ]

In [59]:
# from langchain.text_splitter import RecursiveCharacterTextSplitter
# from transformers import AutoTokenizer


# def split_documents(
#     chunk_size: int,
#     knowledge_base: List[LangchainDocument],
#     tokenizer_name: str,
# ) -> List[LangchainDocument]:
#     """
#     Split documents into chunks of size `chunk_size` characters and return a list of documents.
#     """
#     text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
#         AutoTokenizer.from_pretrained(tokenizer_name),
#         chunk_size=chunk_size,
#         chunk_overlap=int(chunk_size / 10),
#         add_start_index=True,
#         strip_whitespace=True,
#         separators=["\n\n", "\n", ".", " ", ""],
#     )

#     docs_processed = []
#     for doc in knowledge_base:
#         docs_processed += text_splitter.split_documents([doc])

#     # Remove duplicates
#     unique_texts = {}
#     docs_processed_unique = []
#     for doc in docs_processed:
#         if doc.page_content not in unique_texts:
#             unique_texts[doc.page_content] = True
#             docs_processed_unique.append(doc)

#     return docs_processed_unique

### 2.2. Retriever - embeddings 🗂️
The __retriever acts like an internal search engine__: given the user query, it returns the most relevant documents from your knowledge base.

> For the knowledge base, we use Langchain vector databases since __it offers a convenient [FAISS](https://github.com/facebookresearch/faiss) index and allows us to keep document metadata throughout the processing__.

🛠️ __Options included:__

- Tune the chunking method:
    - Size of the chunks
    - Method: split on different separators, use [semantic chunking](https://python.langchain.com/docs/modules/data_connection/document_transformers/semantic-chunker)...
- Change the embedding model

In [60]:
# from langchain.vectorstores import FAISS
# from langchain_community.embeddings import HuggingFaceEmbeddings
# from langchain_community.vectorstores.utils import DistanceStrategy
# import os


# def load_embeddings(
#     langchain_docs: List[LangchainDocument],
#     chunk_size: int,
#     embedding_model_name: Optional[str] = "thenlper/gte-small",
# ) -> FAISS:
#     """
#     Creates a FAISS index from the given embedding model and documents. Loads the index directly if it already exists.

#     Args:
#         langchain_docs: list of documents
#         chunk_size: size of the chunks to split the documents into
#         embedding_model_name: name of the embedding model to use

#     Returns:
#         FAISS index
#     """
#     # load embedding_model
#     embedding_model = HuggingFaceEmbeddings(
#         model_name=embedding_model_name,
#         multi_process=True,
#         model_kwargs={"device": "cuda"},
#         encode_kwargs={
#             "normalize_embeddings": True
#         },  # set True to compute cosine similarity
#     )

#     # Check if embeddings already exist on disk
#     index_name = (
#         f"index_chunk:{chunk_size}_embeddings:{embedding_model_name.replace('/', '~')}"
#     )
#     index_folder_path = f"./data/indexes/{index_name}/"
#     if os.path.isdir(index_folder_path):
#         return FAISS.load_local(
#             index_folder_path,
#             embedding_model,
#             distance_strategy=DistanceStrategy.COSINE,
#         )

#     else:
#         print("Index not found, generating it...")
#         docs_processed = split_documents(
#             chunk_size,
#             langchain_docs,
#             embedding_model_name,
#         )
#         knowledge_index = FAISS.from_documents(
#             docs_processed, embedding_model, distance_strategy=DistanceStrategy.COSINE
#         )
#         knowledge_index.save_local(index_folder_path)
#         return knowledge_index

### 2.3. Reader - LLM 💬

In this part, the __LLM Reader reads the retrieved documents to formulate its answer.__

🛠️ Here we tried the following options to improve results:
- Switch reranking on/off
- Change the reader model

TODO: Already have Mixtral, use it

In [4]:
from langchain.embeddings import CacheBackedEmbeddings, HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
store = LocalFileStore("./cache/")

embed_model_id = 'sentence-transformers/all-MiniLM-L6-v2'

core_embeddings_model = HuggingFaceEmbeddings(
    model_name=embed_model_id
)
embedder = CacheBackedEmbeddings.from_bytes_store(
    core_embeddings_model, store, namespace=embed_model_id
)

vector_store = FAISS.load_local('../rag_index_dir', embedder,allow_dangerous_deserialization=True)

In [9]:
RAG_PROMPT_TEMPLATE = """
<|system|>
Using the information contained in the context,
give a comprehensive answer to the question.
Respond only to the question asked, response should be concise and relevant to the question.
Provide the number of the source document when relevant.
If the answer cannot be deduced from the context, do not give an answer.</s>
<|user|>
Context:
{context}
---
Now here is the question you need to answer.

Question: {question}
</s>
<|assistant|>
"""

In [6]:
from exllamav2 import *
from exllamav2.generator import *
import sys, torch


reader_config = ExLlamaV2Config()
reader_config.model_dir = "/home/mainuser/Desktop/LLMs/ZephyrInference"
#reader_config.model_dir = '/home/mainuser/Desktop/LLMs/Mixtral4bit'
reader_config.prepare()

reader_model = ExLlamaV2(reader_config)
cache = ExLlamaV2Cache(reader_model, lazy = True)

print("Loading model...")
reader_model.load_autosplit(cache)

reader_tokenizer = ExLlamaV2Tokenizer(reader_config)
reader_llm = ExLlamaV2StreamingGenerator(reader_model, cache, reader_tokenizer)
#reader_llm.set_stop_conditions([reader_tokenizer.eos_token_id])
reader_settings = ExLlamaV2Sampler.Settings()
reader_settings.temperature = 0.85
reader_settings.top_k = 30
reader_settings.top_p = 0.8
reader_settings.token_repetition_penalty = 1.03

Loading model...


In [4]:
import os


#os.environ['HUGGINGFACEHUB_API_TOKEN'] = 'xxx' # added to .bashrc, should be good on next restart

In [13]:
from ragatouille import RAGPretrainedModel
from typing import Optional, List, Tuple
RERANKER = RAGPretrainedModel.from_pretrained("colbert-ir/colbertv2.0")
from langchain.docstore.document import Document as LangchainDocument
def answer_with_rag(
    question: str,
    generator: ExLlamaV2StreamingGenerator,
   # tokenizer: ExLlamaV2Tokenizer,
    settings:ExLlamaV2Sampler.Settings,
    max_new_tokens = 512,
    knowledge_index: FAISS = vector_store,
    reranker: Optional[RAGPretrainedModel] = None,
    num_retrieved_docs: int = 10, #30,
    #num_docs_final: int = 5,
) -> Tuple[str, List[LangchainDocument]]:
    # Gather documents with retriever
    print("=> Retrieving documents...")
    embedding_vector = core_embeddings_model.embed_query(question)
    relevant_docs = knowledge_index.similarity_search_by_vector(embedding_vector, k = num_retrieved_docs)#num_retrieved_docs)
    relevant_docs = [doc.page_content for doc in relevant_docs]  # keep only the text

    # Optionally rerank results
    # if reranker:
    #     print("=> Reranking documents...")
    #     relevant_docs = reranker.rerank(question, relevant_docs, k=num_docs_final)
    #     #print(f"Type is : {type(relevant_docs[0])}")
    #     print(dir(relevant_docs[0]))
    #     relevant_docs = [doc['page_content'] for doc in relevant_docs]

    relevant_docs = relevant_docs[:num_retrieved_docs]

    # Build the final prompt
    context = "\nExtracted documents:\n"
    context += "".join([f"Document {str(i)}:::\n" + doc for i, doc in enumerate(relevant_docs)])

   
    generator.warmup()
    final_prompt = RAG_PROMPT_TEMPLATE.format(question=question, context=context)

    answer = generator.generate_simple(final_prompt, 
    settings, max_new_tokens, seed = 1234)
    # print(answer)
    return answer,relevant_docs


answer, relevant_docs = answer_with_rag(question="What is the difference between RAG and self-RAG?", generator=reader_llm,settings=reader_settings,max_new_tokens=512)

=> Retrieving documents...


In [14]:
answer

'\n<|system|>\nUsing the information contained in the context,\ngive a comprehensive answer to the question.\nRespond only to the question asked, response should be concise and relevant to the question.\nProvide the number of the source document when relevant.\nIf the answer cannot be deduced from the context, do not give an answer.</s>\n<|user|>\nContext:\n\nExtracted documents:\nDocument 0:::\nRetrieval Threshold0.60.81.0AccuracyPopQA0.00.51.0\nFrequency\n0.250.500.751.00\nFrequency\n (c) Retrieval\nFigure 3: Analysis on SELF-RAG:(a)Ablation studies for key components of SELF-RAGtraining\nand inference based on our 7B model. (b) Effects of soft weights on ASQA citation precision and\nMauve (fluency). (c) Retrieval frequency andnormalized accuracy on PubHealth and PopQA.\nprecisely grounded yet shorter outputs. Llama2-FT 7B, which is the baseline LM trained on the same\ninstruction-output pairs as SELF-RAGwithout retrieval or self-reflection and is retrieval-augmented\nat test time on

- OK, Zephyr seems to work well, under 4s/question with exl2.  Will try to setup reranker, then onto generating questions and relevant docs.

In [17]:
from ragatouille import RAGPretrainedModel
from typing import Optional, List, Tuple
RERANKER = RAGPretrainedModel.from_pretrained("colbert-ir/colbertv2.0")
from langchain.docstore.document import Document as LangchainDocument
def answer_with_rag(
    question: str,
    generator: ExLlamaV2StreamingGenerator,
   # tokenizer: ExLlamaV2Tokenizer,
    settings:ExLlamaV2Sampler.Settings,
    max_new_tokens = 512,
    knowledge_index: FAISS = vector_store,
    reranker: Optional[RAGPretrainedModel] = None,
    num_retrieved_docs: int = 10, #30,
    num_docs_final: int = 5,
) -> Tuple[str, List[LangchainDocument]]:
    # Gather documents with retriever
    print("=> Retrieving documents...")
    embedding_vector = core_embeddings_model.embed_query(question)
    relevant_docs = knowledge_index.similarity_search_by_vector(embedding_vector, k = num_retrieved_docs)#num_retrieved_docs)
    relevant_docs = [doc.page_content for doc in relevant_docs]  # keep only the text


    if reranker:
        relevant_docs = reranker.rerank(question, relevant_docs, k=num_docs_final)
        relevant_docs = [doc["content"] for doc in relevant_docs]


    relevant_docs = relevant_docs[:num_retrieved_docs]

    # Build the final prompt
    context = "\nExtracted documents:\n"
    context += "".join([f"Document {str(i)}:::\n" + doc for i, doc in enumerate(relevant_docs)])

   
    generator.warmup()
    final_prompt = RAG_PROMPT_TEMPLATE.format(question=question, context=context)

    answer = generator.generate_simple(final_prompt, 
    settings, max_new_tokens, seed = 1234)
    return answer,relevant_docs


answer, relevant_docs = answer_with_rag(question="What is the difference between RAG and self-RAG?", generator=reader_llm,settings=reader_settings,max_new_tokens=512,reranker = RERANKER)

=> Retrieving documents...


100%|██████████| 1/1 [00:00<00:00, 25.24it/s]


In [18]:
answer

"\n<|system|>\nUsing the information contained in the context,\ngive a comprehensive answer to the question.\nRespond only to the question asked, response should be concise and relevant to the question.\nProvide the number of the source document when relevant.\nIf the answer cannot be deduced from the context, do not give an answer.</s>\n<|user|>\nContext:\n\nExtracted documents:\nDocument 0:::\nSELF-RAGis a framework that enhances the quality and factuality of an LLM through retrieval and\nself-reflection, without sacrificing LLM’s original creativity and versatility. Our end-to-end training\nlets an LM Mgenerate text informed by retrieved passages, if needed, and criticize the output by\nlearning to generate special tokens. These reflection tokens (Table 1) signal the need for retrieval\nor confirm the output’s relevance, support, or completeness. In contrast, common RAG approaches\nretrieve passages indiscriminately, without ensuring complete support from cited sources.\n3.1 P ROBLE

In [72]:
from ragatouille import RAGPretrainedModel
from langchain_core.vectorstores import VectorStore
from langchain_core.language_models.llms import LLM




def answer_with_rag(
    question: str,
    llm: LLM,
    knowledge_index: VectorStore,
    reranker: Optional[RAGPretrainedModel] = None,
    num_retrieved_docs: int = 30,
    num_docs_final: int = 7,
) -> Tuple[str, List[LangchainDocument]]:
    """Answer a question using RAG with the given knowledge index."""
    # Gather documents with retriever
    relevant_docs = knowledge_index.similarity_search(
        query=question, k=num_retrieved_docs
    )
    relevant_docs = [doc.page_content for doc in relevant_docs]  # keep only the text

    # Optionally rerank results
    if reranker:
        relevant_docs = reranker.rerank(question, relevant_docs, k=num_docs_final)
        relevant_docs = [doc["content"] for doc in relevant_docs]

    relevant_docs = relevant_docs[:num_docs_final]

    # Build the final prompt
    context = "\nExtracted documents:\n"
    context += "".join(
        [f"Document {str(i)}:::\n" + doc for i, doc in enumerate(relevant_docs)]
    )

    final_prompt = RAG_PROMPT_TEMPLATE.format(question=question, context=context)

    # Redact an answer
    answer = llm(final_prompt)

    return answer, relevant_docs

# 3. Benchmarking the RAG system

The RAG system and the evaluation datasets are now ready. The last step is to judge the RAG system's output on this evlauation dataset.

To this end, __we setup a judge agent__. ⚖️🤖

Out of [the different RAG evaluation metrics](https://docs.ragas.io/en/latest/concepts/metrics/index.html), we choose to focus only on faithfulness since it the best end-to-end metric of our system's performance.

> We use GPT4 as a judge for its empirically good performance, but you could try with other models such as [kaist-ai/prometheus-13b-v1.0](https://huggingface.co/kaist-ai/prometheus-13b-v1.0) or [BAAI/JudgeLM-33B-v1.0](https://huggingface.co/BAAI/JudgeLM-33B-v1.0).

💡 _In the evaluation prompt, we give a detailed description each metric on the scale 1-5, as is done in [Prometheus's prompt template](https://huggingface.co/kaist-ai/prometheus-13b-v1.0): this helps the model ground its metric precisely. If instead you give the judge LLM a vague scale to work with, the outputs will not be consistent enough between different examples._

💡 _Again, prompting the LLM to output rationale before giving its final score gives it more tokens to help it formalize and elaborate a judgement._

In [ ]:
'kaist-ai/prometheus-13b-v1.0'

In [ ]:
def run_rag_tests(
    eval_dataset: datasets.Dataset,
    llm: BaseChatModel,
    knowledge_index: VectorStore,
    output_file: str,
    reranker: Optional[RAGPretrainedModel] = None,
    verbose: Optional[bool] = True,
    test_settings: Optional[str] = None,  # To document the test settings used
):
    """Runs RAG tests on the given dataset and saves the results to the given output file."""
    try:  # load previous generations if they exist
        with open(output_file, "r") as f:
            outputs = json.load(f)
    except:
        outputs = []

    for example in tqdm(eval_dataset):
        question = example["question"]
        if question in [output["question"] for output in outputs]:
            continue

        answer, relevant_docs = answer_with_rag(
            question, llm, knowledge_index, reranker=reranker
        )
        if verbose:
            print("=======================================================")
            print(f"Question: {question}")
            print(f"Answer: {answer}")
            print(f'True answer: {example["answer"]}')
        result = {
            "question": question,
            "true_answer": example["answer"],
            "source_doc": example["source_doc"],
            "generated_answer": answer,
            "retrieved_docs": [doc for doc in relevant_docs],
        }
        if test_settings:
            result["test_settings"] = test_settings
        outputs.append(result)

        with open(output_file, "w") as f:
            json.dump(outputs, f)

In [ ]:
EVALUATION_PROMPT = """###Task Description:
An instruction (might include an Input inside it), a response to evaluate, a reference answer that gets a score of 5, and a score rubric representing a evaluation criteria are given.
1. Write a detailed feedback that assess the quality of the response strictly based on the given score rubric, not evaluating in general.
2. After writing a feedback, write a score that is an integer between 1 and 5. You should refer to the score rubric.
3. The output format should look as follows: \"Feedback: {{write a feedback for criteria}} [RESULT] {{an integer number between 1 and 5}}\"
4. Please do not generate any other opening, closing, and explanations. Be sure to include [RESULT] in your output.

###The instruction to evaluate:
{instruction}

###Response to evaluate:
{response}

###Reference Answer (Score 5):
{reference_answer}

###Score Rubrics:
[Is the response correct, accurate, and factual based on the reference answer?]
Score 1: The response is completely incorrect, inaccurate, and/or not factual.
Score 2: The response is mostly incorrect, inaccurate, and/or not factual.
Score 3: The response is somewhat correct, accurate, and/or factual.
Score 4: The response is mostly correct, accurate, and factual.
Score 5: The response is completely correct, accurate, and factual.

###Feedback:"""

from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import SystemMessage


evaluation_prompt_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content="You are a fair evaluator language model."),
        HumanMessagePromptTemplate.from_template(EVALUATION_PROMPT),
    ]
)

In [75]:
#from langchain.chat_models import ChatOpenAI

#eval_chat_model = ChatOpenAI(model="gpt-4-1106-preview", temperature=0)
eval_chat_repo = 'kaist-ai/prometheus-13b-v1.0'
evaluator_name = "PROMETHEUS-13B"

repo_id = eval_chat_repo
READER_MODEL_NAME = "kaist-ai/prometheus-13b-v1.0"

eval_chat_model = HuggingFaceHub(
    repo_id=repo_id,
    task="text-generation",
    model_kwargs={
        "max_new_tokens": 512,
        "top_k": 30,
        "temperature": 0.1,
        "repetition_penalty": 1.03,
    },
)



def evaluate_answers(
    answer_path: str,
    eval_chat_model,#BaseChatModel,
    evaluator_name: str,
    evaluation_prompt_template #: ChatPromptTemplate,
) -> None:
    """Evaluates generated answers. Modifies the given answer file in place for better checkpointing."""
    answers = []
    if os.path.isfile(answer_path):  # load previous generations if they exist
        answers = json.load(open(answer_path, "r"))

    for experiment in tqdm(answers):
        if f"eval_score_{evaluator_name}" in experiment:
            continue

        eval_prompt = evaluation_prompt_template.format_messages(
            instruction=experiment["question"],
            response=experiment["generated_answer"],
            reference_answer=experiment["true_answer"],
        )
        eval_result = eval_chat_model.invoke(eval_prompt)
        feedback, score = [
            item.strip() for item in eval_result.content.split("[RESULT]")
        ]
        experiment[f"eval_score_{evaluator_name}"] = score
        experiment[f"eval_feedback_{evaluator_name}"] = feedback

        with open(answer_path, "w") as f:
            json.dump(answers, f)

🚀 Let's run the tests and evaluate answers!👇

In [ ]:
if not os.path.exists("./output"):
    os.mkdir("./output")

for chunk_size in [200]:  # Add other chunk sizes (in tokens) as needed
    for embeddings in ["thenlper/gte-small"]:  # Add other embeddings as needed
        for rerank in [True, False]:
            settings_name = f"chunk:{chunk_size}_embeddings:{embeddings.replace('/', '~')}_rerank:{rerank}_reader-model:{READER_MODEL_NAME}"
            output_file_name = f"./output/rag_{settings_name}.json"

            print(f"Running evaluation for {settings_name}:")

            print("Loading knowledge base embeddings...")
            knowledge_index = load_embeddings(
                RAW_KNOWLEDGE_BASE,
                chunk_size=chunk_size,
                embedding_model_name=embeddings,
            )

            print("Running RAG...")
            reranker = (
                RAGPretrainedModel.from_pretrained("colbert-ir/colbertv2.0")
                if rerank
                else None
            )
            run_rag_tests(
                eval_dataset=eval_dataset,
                llm=READER_LLM,
                knowledge_index=knowledge_index,
                output_file=output_file_name,
                reranker=reranker,
                verbose=False,
                test_settings=settings_name,
            )

            print("Running evaluation...")
            evaluate_answers(
                output_file_name,
                eval_chat_model,
                evaluator_name,
                evaluation_prompt_template,
            )

### Inspect results

In [ ]:
import glob

outputs = []
for file in glob.glob("./output/*.json"):
    output = pd.DataFrame(json.load(open(file, "r")))
    output["settings"] = file
    outputs.append(output)
result = pd.concat(outputs)

In [ ]:
result["eval_score_GPT4"] = result["eval_score_GPT4"].apply(
    lambda x: int(x) if isinstance(x, str) else 1
)
result["eval_score_GPT4"] = (result["eval_score_GPT4"] - 1) / 4

In [ ]:
average_scores = result.groupby("settings")["eval_score_GPT4"].mean()
average_scores.sort_values()

settings
./output/rag_chunk:200_embeddings:thenlper~gte-small_rerank:False_reader-model:zephyr-7b-beta.json       0.884328
./output/rag_chunk:200_embeddings:BAAI~bge-base-en-v1.5_rerank:False_reader-model:zephyr-7b-beta.json    0.906716
./output/rag_chunk:200_embeddings:BAAI~bge-base-en-v1.5_rerank:True_reader-model:zephyr-7b-beta.json     0.906716
./output/rag_chunk:200_embeddings:thenlper~gte-small_rerank:True_reader-model:mixtral.json               0.906716
./output/rag_chunk:200_embeddings:thenlper~gte-small_rerank:True_reader-model:zephyr-7b-beta.json        0.921642
./output/rag_chunk:200_embeddings:thenlper~gte-small_rerank:True_reader-model:mixtral0.json              0.947761
Name: eval_score_GPT4, dtype: float64

## Example results

Let us load the results that I obtained by tweaking the different options available in this notebook.
For more detail on why these options could work on not, see the notebook on [advanced_RAG](advanced_rag).

As you can see in the graph below, some tweaks do not bring any improvement, some give huge performance boosts.

➡️ ___There is no single good recipe: you should try several different directions when tuning your RAG systems.___


In [ ]:
import plotly.express as px

scores = datasets.load_dataset("m-ric/rag_scores_cookbook", split="train")
scores = pd.Series(scores["score"], index=scores["settings"])

In [ ]:
fig = px.bar(
    scores,
    color=scores,
    labels={
        "value": "Accuracy",
        "settings": "Configuration",
    },
    color_continuous_scale="bluered",
)
fig.update_layout(w
    width=1000,
    height=600,
    barmode="group",
    yaxis_range=[0, 100],
    title="<b>Accuracy of different RAG configurations</b>",
    xaxis_title="RAG settings",
    font=dict(size=15),
)
fig.layout.yaxis.ticksuffix = "%"
fig.update_coloraxes(showscale=False)
fig.update_traces(texttemplate="%{y:.1f}", textposition="outside")
fig.show()